In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
generator = 0

In [3]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\selfie2anime\trainB"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            if(np.array(x).shape == (128,128,3)):
                data.append(np.array(x))

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),128,128,3)
        return x_train

In [4]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 200

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [5]:
dcgan = DCGAN()
dcgan.train(epochs=10001, batch_size=40, save_interval=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 33, 33, 64)       

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.902577, acc.: 28.75%] [G loss: 0.302418]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.498372, acc.: 68.75%] [G loss: 2.764158]
2 [D loss: 1.001543, acc.: 50.00%] [G loss: 1.072332]
3 [D loss: 0.496419, acc.: 76.25%] [G loss: 1.280170]
4 [D loss: 0.264196, acc.: 96.25%] [G loss: 1.189412]
5 [D loss: 0.242406, acc.: 93.75%] [G loss: 1.115900]
6 [D loss: 0.270863, acc.: 91.25%] [G loss: 1.080159]
7 [D loss: 0.388042, acc.: 85.00%] [G loss: 1.498522]
8 [D loss: 0.397755, acc.: 86.25%] [G loss: 1.402995]
9 [D loss: 0.280078, acc.: 93.75%] [G loss: 1.545541]
10 [D loss: 0.648222, acc.: 57.50%] [G loss: 3.032286]
11 [D loss: 0.790510, acc.: 50.00%] [G loss: 2.678479]
12 [D loss: 0.926234, acc.: 46.25%] [G loss: 2.646232]
13 [D loss: 0.280488, acc.: 90.00%] [G loss: 3.707854]
14 [D loss: 0.505002, acc.: 80.00%] [G loss: 2.624522]
15 [D loss: 0.254156, acc.: 91.25%] [G loss: 4.112914]
16 [D loss: 0.154411, acc.: 95.00%] [G loss: 3.346097]
17 [D loss: 0.342336, acc.: 88.75%] [G loss: 2.911653]
18 [D loss: 0.260372, acc.: 91.25%] [G loss: 3.390109]
19 [D loss: 0.15886

...

9383 [D loss: 0.208198, acc.: 91.25%] [G loss: 4.843076]
9384 [D loss: 0.235880, acc.: 90.00%] [G loss: 7.186687]
9385 [D loss: 0.108432, acc.: 97.50%] [G loss: 6.230271]
9386 [D loss: 0.305419, acc.: 83.75%] [G loss: 6.163475]
9387 [D loss: 0.177453, acc.: 90.00%] [G loss: 4.910661]
9388 [D loss: 0.149100, acc.: 93.75%] [G loss: 7.738755]
9389 [D loss: 0.714673, acc.: 70.00%] [G loss: 6.371984]
9390 [D loss: 0.122830, acc.: 95.00%] [G loss: 5.797932]
9391 [D loss: 0.494956, acc.: 78.75%] [G loss: 7.457213]
9392 [D loss: 0.119673, acc.: 97.50%] [G loss: 6.154197]
9393 [D loss: 0.225233, acc.: 91.25%] [G loss: 4.748296]
9394 [D loss: 0.131677, acc.: 93.75%] [G loss: 7.993955]
9395 [D loss: 0.463290, acc.: 72.50%] [G loss: 6.284644]
9396 [D loss: 0.217504, acc.: 87.50%] [G loss: 5.935523]
9397 [D loss: 0.480534, acc.: 78.75%] [G loss: 7.595802]
9398 [D loss: 0.412917, acc.: 88.75%] [G loss: 4.145225]
9399 [D loss: 0.204688, acc.: 91.25%] [G loss: 6.111981]
9400 [D loss: 0.343853, acc.: 8

9527 [D loss: 0.063650, acc.: 98.75%] [G loss: 5.672500]
9528 [D loss: 0.161880, acc.: 91.25%] [G loss: 5.450052]
9529 [D loss: 0.171881, acc.: 92.50%] [G loss: 6.127767]
9530 [D loss: 0.393169, acc.: 82.50%] [G loss: 7.355215]
9531 [D loss: 0.496630, acc.: 78.75%] [G loss: 5.900504]
9532 [D loss: 0.067478, acc.: 97.50%] [G loss: 5.571236]
9533 [D loss: 0.246595, acc.: 88.75%] [G loss: 6.109395]
9534 [D loss: 0.184126, acc.: 92.50%] [G loss: 5.605019]
9535 [D loss: 0.107794, acc.: 95.00%] [G loss: 5.883627]
9536 [D loss: 0.097914, acc.: 96.25%] [G loss: 5.806572]
9537 [D loss: 0.252859, acc.: 90.00%] [G loss: 7.124971]
9538 [D loss: 0.575049, acc.: 71.25%] [G loss: 7.887992]
9539 [D loss: 0.234567, acc.: 92.50%] [G loss: 4.761701]
9540 [D loss: 0.288483, acc.: 87.50%] [G loss: 4.443212]
9541 [D loss: 0.162128, acc.: 93.75%] [G loss: 6.097457]
9542 [D loss: 0.142359, acc.: 93.75%] [G loss: 5.151537]
9543 [D loss: 0.232429, acc.: 88.75%] [G loss: 4.605634]
9544 [D loss: 0.279671, acc.: 9

9671 [D loss: 0.202992, acc.: 91.25%] [G loss: 5.060132]
9672 [D loss: 0.213914, acc.: 88.75%] [G loss: 7.541898]
9673 [D loss: 0.662484, acc.: 70.00%] [G loss: 4.911609]
9674 [D loss: 0.148721, acc.: 92.50%] [G loss: 6.613706]
9675 [D loss: 0.243391, acc.: 90.00%] [G loss: 5.936720]
9676 [D loss: 0.272440, acc.: 87.50%] [G loss: 8.466876]
9677 [D loss: 0.361876, acc.: 88.75%] [G loss: 3.159760]
9678 [D loss: 0.200790, acc.: 92.50%] [G loss: 6.183800]
9679 [D loss: 0.250662, acc.: 87.50%] [G loss: 7.212170]
9680 [D loss: 0.591785, acc.: 77.50%] [G loss: 7.612714]
9681 [D loss: 0.504969, acc.: 77.50%] [G loss: 4.356654]
9682 [D loss: 0.062671, acc.: 97.50%] [G loss: 4.774278]
9683 [D loss: 0.350291, acc.: 85.00%] [G loss: 7.324268]
9684 [D loss: 0.129646, acc.: 93.75%] [G loss: 5.220431]
9685 [D loss: 0.147031, acc.: 95.00%] [G loss: 5.849509]
9686 [D loss: 0.216533, acc.: 91.25%] [G loss: 4.457472]
9687 [D loss: 0.279536, acc.: 85.00%] [G loss: 6.456328]
9688 [D loss: 0.165390, acc.: 9

9815 [D loss: 0.100499, acc.: 93.75%] [G loss: 4.724027]
9816 [D loss: 0.173672, acc.: 95.00%] [G loss: 4.783953]
9817 [D loss: 0.128188, acc.: 96.25%] [G loss: 6.206212]
9818 [D loss: 0.049151, acc.: 100.00%] [G loss: 5.298016]
9819 [D loss: 0.161067, acc.: 96.25%] [G loss: 6.176690]
9820 [D loss: 0.264850, acc.: 88.75%] [G loss: 5.931117]
9821 [D loss: 0.250830, acc.: 85.00%] [G loss: 4.642861]
9822 [D loss: 0.093636, acc.: 96.25%] [G loss: 5.779934]
9823 [D loss: 0.482385, acc.: 83.75%] [G loss: 6.809325]
9824 [D loss: 0.175650, acc.: 96.25%] [G loss: 6.328546]
9825 [D loss: 0.181893, acc.: 92.50%] [G loss: 6.609084]
9826 [D loss: 0.159173, acc.: 95.00%] [G loss: 6.231734]
9827 [D loss: 0.263055, acc.: 91.25%] [G loss: 5.165493]
9828 [D loss: 0.203225, acc.: 90.00%] [G loss: 5.722069]
9829 [D loss: 0.149839, acc.: 91.25%] [G loss: 5.904232]
9830 [D loss: 0.104831, acc.: 97.50%] [G loss: 5.744700]
9831 [D loss: 0.136368, acc.: 93.75%] [G loss: 5.590450]
9832 [D loss: 0.111768, acc.: 

9959 [D loss: 0.154373, acc.: 92.50%] [G loss: 5.279929]
9960 [D loss: 0.140970, acc.: 93.75%] [G loss: 6.698906]
9961 [D loss: 0.269787, acc.: 87.50%] [G loss: 4.442155]
9962 [D loss: 0.156546, acc.: 93.75%] [G loss: 6.791784]
9963 [D loss: 0.186926, acc.: 95.00%] [G loss: 5.103608]
9964 [D loss: 0.305940, acc.: 86.25%] [G loss: 7.219357]
9965 [D loss: 0.197995, acc.: 90.00%] [G loss: 7.008726]
9966 [D loss: 0.425963, acc.: 83.75%] [G loss: 5.412196]
9967 [D loss: 0.137658, acc.: 93.75%] [G loss: 6.601254]
9968 [D loss: 0.300296, acc.: 87.50%] [G loss: 4.666961]
9969 [D loss: 0.130687, acc.: 96.25%] [G loss: 6.539551]
9970 [D loss: 0.391501, acc.: 82.50%] [G loss: 6.321385]
9971 [D loss: 0.136507, acc.: 93.75%] [G loss: 6.045104]
9972 [D loss: 0.414517, acc.: 81.25%] [G loss: 5.343572]
9973 [D loss: 0.079800, acc.: 98.75%] [G loss: 5.886827]
9974 [D loss: 0.362621, acc.: 86.25%] [G loss: 6.103058]
9975 [D loss: 0.327627, acc.: 85.00%] [G loss: 5.509948]
9976 [D loss: 0.162526, acc.: 9